In [26]:
import pandas as pd
import re
from datasets import Dataset

# Load Dataset

In [6]:
data_path = "../data/naruto.csv"
naruto_transcript_df = pd.read_csv(data_path)

In [7]:
naruto_transcript_df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [9]:
# Remove actions from transcript
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)','',text)
    return result

In [11]:
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)

In [13]:
naruto_transcript_df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [14]:
naruto_transcript_df['number_of_words'] = naruto_transcript_df['line'].str.strip().str.split(" ")
naruto_transcript_df['number_of_words'] = naruto_transcript_df['number_of_words'].apply(lambda x: len(x))

In [15]:
naruto_transcript_df.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [16]:
naruto_transcript_df['naruto_response_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name']=="Naruto")&(naruto_transcript_df['number_of_words']>5),'naruto_response_flag']=1

In [17]:
naruto_transcript_df

,name,line,number_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",26,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0
...,...,...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...,27,0
159,Iruka,Huh?,1,0
160,Naruto,Iruka Sensei!,2,0
161,Iruka,Ah! That hurts!,3,0


In [18]:
indexes_to_take = list(naruto_transcript_df[(naruto_transcript_df['naruto_response_flag']==1)&(naruto_transcript_df.index>0)].index)

In [19]:
indexes_to_take[:3]

[6, 28, 30]

In [20]:
system_promt = """" Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n"""

prompts = []
for ind in indexes_to_take:
    prompt = system_promt

    prompt += naruto_transcript_df.iloc[ind -1]['line']
    prompt += '\n'
    prompt += naruto_transcript_df.iloc[ind]['line']
    prompts.append(prompt)

In [24]:
print(prompts[3])

" Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns 
 Well, I was just thinking. Maybe after you clean this all up I can take you out for some ramen. The good stuff. What do you think? 
  Now that’s some serious motivation! I’ll have this clean in no time!


In [25]:
df = pd.DataFrame({"prompt":prompts})
df.head()

,prompt
0,""" Your are Naruto from the anime ""Naruto"". You..."
1,""" Your are Naruto from the anime ""Naruto"". You..."
2,""" Your are Naruto from the anime ""Naruto"". You..."
3,""" Your are Naruto from the anime ""Naruto"". You..."
4,""" Your are Naruto from the anime ""Naruto"". You..."


In [27]:
dataset = Dataset.from_pandas(df)